<a href="https://colab.research.google.com/github/mukeshrock7897/GenerativeAI/blob/main/GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generative AI**

# **Beginner Level**

1. Introduction to Generative AI
    * What is Generative AI?
    * History and evolution
    * Key concepts and terminology

2. Types of Generative Models
    * Overview of different generative models
    * Comparison between generative and discriminative models

3. Applications of Generative AI
    * Image generation
    * Text generation
    * Music generation
    * Other applications

4. Ethical and Social Implications
    * Bias and fairness
    * Privacy concerns
    * Ethical use of generative models



# **1.Introduction to Generative AI**

**What is Generative AI?**
* Generative AI refers to the use of artificial intelligence to create new content, such as images, text, music, and more. It involves training models to generate data that is similar to a given dataset.

**History and Evolution**
* Overview of the development of generative AI, from early rule-based systems to modern deep learning models.

**Key Concepts and Terminology**
* Understanding essential terms such as generative models, training data, latent space, and sampling.

**Types of Generative Models**

* Overview of Different Generative Models

   * Introduction to various generative models, including:
     * Generative Adversarial Networks (GANs)
     * Variational Autoencoders (VAEs)
     * Autoregressive Models (e.g., GPT, PixelRNN)
* Comparison Between Generative and Discriminative Models
  * Explanation of the differences between generative models, which generate new data, and discriminative models, which classify data.

**Applications of Generative AI**

**Image Generation**
* Use cases in art, design, and enhancing images. Examples include DeepArt and StyleGAN.

**Text Generation**
* Applications in content creation, summarization, and conversational agents. Examples include GPT-3 and ChatGPT.

**Music Generation**
* Creating music using models like OpenAI’s MuseNet or Jukedeck.

**Other Applications**
* Exploring generative AI in fields such as game design, drug discovery, and

**video generation**
* Ethical and Social Implications

**Bias and Fairness**
* Understanding how generative models can inherit and amplify biases present in training data.

**Privacy Concerns**
* Issues related to the generation of data that can mimic real individuals or sensitive content.

**Ethical Use of Generative Models**
* Discussion on the responsible use of generative AI, including potential misuse and guidelines for ethical deployment.

# **Intermediate Level**

1. Generative Adversarial Networks (GANs)
    * GAN architecture
    * Training GANs
    * Common GAN variants (e.g., DCGAN, CycleGAN)

2. Variational Autoencoders (VAEs)
    * VAE architecture
    * Training VAEs
    * Applications of VAEs

3. Autoregressive Models
    * Overview of autoregressive models
    * PixelCNN and PixelRNN
    * GPT (Generative Pre-trained Transformer)

4. Flow-based Models
    * Introduction to flow-based models
    * Normalizing flows
    * RealNVP and Glow

5. Diffusion Models
    * Basics of diffusion models
    * Denoising diffusion probabilistic models


# **1. Generative Adversarial Networks (GANs)**

**GAN Architecture**
* Generative Adversarial Networks consist of two neural networks: the Generator and the Discriminator. The Generator creates fake data that mimics the real data, while the Discriminator tries to distinguish between real and fake data.

    * **Generator:** Takes a random noise vector and generates data.
    * **Discriminator:** Takes data (real or generated) and predicts whether it is real or fake.

In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x.view(-1, 28*28))


**Training GANs**
* Training involves a two-step process where the Discriminator and Generator are trained alternately. The Discriminator learns to distinguish real from fake data, while the Generator learns to produce more realistic fake data to fool the Discriminator.

In [ ]:
import torch.optim as optim

# Initialize models
G = Generator()
D = Discriminator()

# Loss and optimizers
criterion = nn.BCELoss()
optimizerD = optim.Adam(D.parameters(), lr=0.0002)
optimizerG = optim.Adam(G.parameters(), lr=0.0002)

for epoch in range(10000):
    # Train Discriminator
    real_data = torch.randn(batch_size, 28*28)
    fake_data = G(torch.randn(batch_size, 100)).detach()
    optimizerD.zero_grad()
    lossD = criterion(D(real_data), torch.ones(batch_size, 1)) + \
            criterion(D(fake_data), torch.zeros(batch_size, 1))
    lossD.backward()
    optimizerD.step()

    # Train Generator
    fake_data = G(torch.randn(batch_size, 100))
    optimizerG.zero_grad()
    lossG = criterion(D(fake_data), torch.ones(batch_size, 1))
    lossG.backward()
    optimizerG.step()


**Common GAN Variants**

* **DCGAN (Deep Convolutional GAN):** Uses convolutional layers to improve the quality of generated images.
* **CycleGAN:** Used for image-to-image translation without paired examples.

# **2. Variational Autoencoders (VAEs)**

**VAE Architecture**
* VAEs consist of an encoder that maps input data to a latent space and a decoder that reconstructs the data from the latent space. Unlike traditional autoencoders, VAEs impose a probabilistic structure on the latent space.

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.ReLU(),
            nn.Linear(400, 20)
        )
        self.decoder = nn.Sequential(
            nn.Linear(10, 400),
            nn.ReLU(),
            nn.Linear(400, 28*28),
            nn.Sigmoid()
        )

    def encode(self, x):
        h = self.encoder(x)
        return h[:, :10], h[:, 10:]

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 28*28))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


**Training VAEs**
* The training objective of VAEs is to minimize the reconstruction loss and the Kullback-Leibler divergence between the latent distribution and a prior distribution.

In [ ]:
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 28*28), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    for data in dataloader:
        x, _ = data
        optimizer.zero_grad()
        recon_x, mu, logvar = model(x)
        loss = loss_function(recon_x, x, mu, logvar)
        loss.backward()
        optimizer.step()


**Applications of VAEs**
* Image generation
* Anomaly detection
* Data augmentation


# **3. Autoregressive Models**

**Overview of Autoregressive Models**
* Autoregressive models generate each output step-by-step, conditioned on previous steps.

**Common examples include GPT and PixelCNN.**

**PixelCNN and PixelRNN**
* These models generate images pixel by pixel, where each pixel is conditioned on previously generated pixels.

In [ ]:
class PixelCNN(nn.Module):
    def __init__(self):
        super(PixelCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, padding=3)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=7, padding=3)
        self.conv3 = nn.Conv2d(64, 1, kernel_size=7, padding=3)

    def forward(self, x):
        x = torch.sigmoid(self.conv1(x))
        x = torch.sigmoid(self.conv2(x))
        x = torch.sigmoid(self.conv3(x))
        return x


**GPT (Generative Pre-trained Transformer)**
* GPT models use the transformer architecture and are trained on large text corpora to generate human-like text.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

output = model.generate(input_ids, max_length=50, num_return_sequences=1)
print(tokenizer.decode(output[0], skip_special_tokens=True))

# **4. Flow-based Models**

**Introduction to Flow-based Models**
* Flow-based models use invertible neural networks to transform data into a latent space where it follows a simple distribution.

**Normalizing Flows**
* Normalizing flows are a series of invertible transformations applied to the data to match a target distribution.

In [ ]:
class NormalizingFlow(nn.Module):
    def __init__(self):
        super(NormalizingFlow, self).__init__()
        self.layers = nn.ModuleList([AffineCoupling() for _ in range(4)])

    def forward(self, x):
        log_det_jacobian = 0
        for layer in self.layers:
            x, ldj = layer(x)
            log_det_jacobian += ldj
        return x, log_det_jacobian

    def inverse(self, z):
        for layer in reversed(self.layers):
            z = layer.inverse(z)
        return z


**RealNVP and Glow**
* **RealNVP:** Introduced affine coupling layers for stable training.
* **Glow:** Improved architecture with 1x1 convolutions and better performance.


# **5. Diffusion Models**
**Basics of Diffusion Models**
* Diffusion models gradually add noise to the data during training and learn to reverse this process to generate data.

**Denoising Diffusion Probabilistic Models**
* These models use a series of denoising autoencoders to iteratively refine noisy data until it matches the target distribution.

In [ ]:
class DDPM(nn.Module):
    def __init__(self):
        super(DDPM, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        noise = torch.randn_like(x)
        x_noisy = x + noise
        encoded = self.encoder(x_noisy)
        return self.decoder(encoded), noise

# **Advanced Level**

1. Advanced Topics in Generative AI
    * Advanced GAN architectures (e.g., StyleGAN, BigGAN)
    * Advanced VAE techniques
    * Advanced autoregressive models

2. Practical Implementation and Tools
    * Implementing generative models using frameworks (TensorFlow, PyTorch)
    * Best practices for training generative models
    * Debugging and improving generative models


# **1. Advanced Topics in Generative AI**

# **Advanced GAN Architectures**

**StyleGAN**
* StyleGAN introduces style transfer into GANs by injecting noise at different layers of the generator.

In [ ]:
# Assuming pre-trained StyleGAN model is available
from stylegan2_pytorch import ModelLoader

model = ModelLoader(image_size=256, network_capacity=16)
generated_image = model.generate_from_noise(1)
generated_image.save('stylegan_image.png')


**BigGAN**
* BigGAN scales up GANs to generate high-quality images by using larger batch sizes and better architectural designs.

In [ ]:
# Pseudo code for BigGAN
from biggan import BigGAN, one_hot_from_names, truncated_noise_sample

model = BigGAN.from_pretrained('biggan-deep-256')
truncation = 0.4
class_vector = one_hot_from_names(['dog', 'cat'], batch_size=1)
noise_vector = truncated_noise_sample(truncation=truncation, batch_size=1)
generated_image = model(noise_vector, class_vector, truncation)


# **Advanced VAE Techniques**

**Beta-VAE**
* Beta-VAE introduces a hyperparameter β to balance the trade-off between reconstruction and latent distribution regularization.

In [ ]:
class BetaVAE(nn.Module):
    def __init__(self, beta=1):
        super(BetaVAE, self).__init__()
        self.beta = beta
        # Define encoder and decoder here...

    def forward(self, x):
        # Define forward pass here...
        pass

    def loss_function(self, recon_x, x, mu, logvar):
        BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 28*28), reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return BCE + self.beta * KLD


**VQ-VAE (Vector Quantized VAE)**
* VQ-VAE uses discrete latent variables for representation learning, which is useful for tasks like image generation and speech synthesis.

In [ ]:
class VQVAE(nn.Module):
    def __init__(self, K, D):
        super(VQVAE, self).__init__()
        self.embedding = nn.Embedding(K, D)
        # Define encoder and decoder here...

    def forward(self, x):
        # Define forward pass here...
        pass


# **Advanced Autoregressive Models**

**Transformer-XL**
* Transformer-XL handles long-term dependencies better by segmenting the input sequence and using recurrence.



In [ ]:
from transformers import TransfoXLModel, TransfoXLTokenizer

tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
model = TransfoXLModel.from_pretrained('transfo-xl-wt103')

input_text = "The quick brown fox"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
outputs = model(input_ids)


**Reformer**
* Reformer uses locality-sensitive hashing and reversible layers to efficiently handle long sequences.

In [ ]:
from reformer_pytorch import ReformerLM

model = ReformerLM(
    num_tokens=20000,
    dim=512,
    depth=6,
    max_seq_len=4096,
    heads=8,
    lsh_dropout=0.1,
    causal=True
)

input_text = torch.randint(0, 20000, (1, 4096))
output = model(input_text)


# **2. Practical Implementation and Tools**

**Implementing Generative Models using Frameworks**

**TensorFlow**
* TensorFlow provides extensive support for building and training generative models.

In [ ]:
import tensorflow as tf

class GAN(tf.keras.Model):
    def __init__(self):
        super(GAN, self).__init__()
        self.generator = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(28*28, activation='tanh')
        ])
        self.discriminator = tf.keras.Sequential([
            tf.keras.layers.Dense(512, activation='relu'),
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

    def call(self, inputs):
        generated_data = self.generator(inputs)
        return self.discriminator(generated_data)


**PyTorch**
* PyTorch is popular for its dynamic computational graph and ease of use.

In [ ]:
import torch
import torch.nn as nn

class GAN(nn.Module):
    def __init__(self):
        super(GAN, self).__init__()
        self.generator = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )
        self.discriminator = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.discriminator(self.generator(x))


**Best Practices for Training Generative Models**
* **Hyperparameter tuning:** Systematically adjust hyperparameters for optimal performance.

* **Regularization:** Use techniques like dropout and weight decay to prevent overfitting.
* **Data augmentation:** Enhance training data with transformations to improve model robustness.


**Debugging and Improving Generative Models**
* **Visual inspection:** Regularly visualize generated samples to assess quality.
* **Loss monitoring:** Track generator and discriminator losses to detect issues.
* **Model architecture:** Experiment with different architectures and layer configurations.

# **3.Advanced Topics**

**Conditional Generative Models**
* Conditional generative models generate data conditioned on certain inputs, such as class labels.

In [ ]:
class ConditionalGAN(nn.Module):
    def __init__(self, num_classes):
        super(ConditionalGAN, self).__init__()
        self.generator = nn.Sequential(
            nn.Linear(100 + num_classes, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28*28),
            nn.Tanh()
        )
        self.discriminator = nn.Sequential(
            nn.Linear(28*28 + num_classes, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
        x = torch.cat([x, labels], dim=1)
        return self.discriminator(self.generator(x))


**Meta-Learning and Few-Shot Learning**
* Meta-learning techniques can adapt generative models to new tasks with limited data.

In [ ]:
from torchmeta.datasets.helpers import miniimagenet
from torchmeta.utils.data import BatchMetaDataLoader

dataset = miniimagenet('data', ways=5, shots=1, test_shots=15, meta_train=True, download=True)
dataloader = BatchMetaDataLoader(dataset, batch_size=16, num_workers=4)

for batch in dataloader:
    inputs, targets = batch['train']
    # Model training code here...


**Reinforcement Learning and Generative Models**
* Combining reinforcement learning with generative models can lead to advanced applications like game playing and strategy optimization.

In [ ]:
import gym
from stable_baselines3 import PPO

env = gym.make('CartPole-v1')
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

# **All Libraries and Tools in Generative AI**



# **Python Libraries and Frameworks:**
1. TensorFlow
2. Keras
3. PyTorch
4. Hugging Face Transformers
5. OpenAI GPT-3/GPT-4
6. Fastai
7. GANLab
8. GANMetrics
9. StyleGAN2-ADA
10. BigGAN-PyTorch
11. VQ-VAE-2
12. DiffWave
13. MuseNet
14. DeepArt
15. Magenta
16. Dreamer
17. DeOldify
18. RunwayML
19. OpenCV
20. Pydub
21. Numpy
22. Scipy
23. Matplotlib
24. Seaborn

# **Specialized Tools and Libraries:**
1. DALL-E
2. CLIP
3. DeepDream
4. Jukebox
5. Taming Transformers
6. PPLM (Plug and Play Language Models)
7. Reformer
8. Longformer
9. EleutherAI’s GPT-Neo
10. GPT-J
11. RealNVP
12. Glow
13. Neural Style Transfer (NST)

# **Autoencoders and Variants:**
1. Autoencoder (AE)
2. Variational Autoencoder (VAE)
3. Beta-VAE
4. Conditional VAE (CVAE)
5. Denoising Autoencoder (DAE)

# **GANs and Variants:**
1. Generative Adversarial Networks (GANs)
2. Deep Convolutional GAN (DCGAN)
3. CycleGAN
4. StyleGAN
5. BigGAN
6. Conditional GAN (cGAN)
7. Progressive GAN (PGGAN)
8. Wasserstein GAN (WGAN)
9. Least Squares GAN (LSGAN)

# **Diffusion Models:**
1. Denoising Diffusion Probabilistic Models (DDPM)
2. Score-Based Generative Models

# **Other Models:**
1. PixelCNN
2. PixelRNN
3. GLOW
4. RealNVP

